In [143]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)
import missingno as msno
import os

In [144]:
%%time
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
# 혹시 코랩환경을 사용하신다면 왼쪽 폴더모양 아이콘을 눌러 "train_data.csv"를 드래그&드롭으로 업로드한 후 사용해주세요
DATA_PATH = '../../../data/train_data.csv'
DATA_PATH_T = '../../../data/test_data.csv'
df_train = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
df_train = df_train.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
df_test = pd.read_csv(DATA_PATH_T, dtype=dtype, parse_dates=['Timestamp'])
df_test = df_test.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

CPU times: user 4.31 s, sys: 292 ms, total: 4.6 s
Wall time: 4.61 s


In [145]:
data=pd.concat([df_train,df_test])
data.drop_duplicates(subset=["userID", "assessmentItemID"], keep="last", inplace=True)

train_data = data[data.answerCode > 0]
test_data = data[data.answerCode < 0]

userid, itemid = (
    sorted(list(set(data.userID))),
    sorted(list(set(data.assessmentItemID))),
)
n_user, n_item = len(userid), len(itemid)

# userid_2index, itemid_2_index는 text로 저장할 파일임
userid_2_index = {v: i for i, v in enumerate(userid)}
itemid_2_index = {v: i for i, v in enumerate(itemid)}


In [146]:
train_data.drop(['testId','answerCode','Timestamp','KnowledgeTag'],axis=1)

,userID,assessmentItemID
0,0,A060001001
1,0,A060001002
2,0,A060001003
3,0,A060001004
4,0,A060001005
...,...,...
260105,7439,A040197003
260108,7439,A040197006
260110,7439,A040130002
260111,7439,A040130003


In [147]:
test_data.drop(['testId','answerCode','Timestamp','KnowledgeTag'],axis=1)

,userID,assessmentItemID
1035,3,A050133008
1706,4,A070146008
3023,13,A070111008
4283,17,A090064006
4670,26,A060135007
...,...,...
260052,7395,A040122005
260067,7404,A030111005
260082,7416,A050193004
260097,7417,A050193004


In [148]:
user_list=pd.DataFrame(list(userid_2_index.items()),columns=['org_id', 'remap_id'])
item_list=pd.DataFrame(list(itemid_2_index.items()),columns=['org_id', 'remap_id'])
user_list.to_csv('user_list.txt', sep = ' ', index=False)
item_list.to_csv('item_list.txt', sep = ' ', index=False)

In [149]:
# train_data dic 만들기
train_data_dic={}
for index, row in train_data.iterrows():
    if row['userID'] in train_data_dic:
        train_data_dic[row['userID']].append(itemid_2_index[row['assessmentItemID']])
    else:
        train_data_dic[row['userID']]=[itemid_2_index[row['assessmentItemID']]]
with open('train.txt','w',encoding='UTF-8') as f:
    for code,name in train_data_dic.items():
        f.write(f'{code}')
        for i in range(len(name)):
            f.write(f' {name[i]}')
        f.write(f'\n')

In [92]:
# test_data dic 만들기
test_data_dic={}
for index, row in test_data.iterrows():
    if row['userID'] in test_data_dic:
        test_data_dic[row['userID']].append(itemid_2_index[row['assessmentItemID']])
    else:
        test_data_dic[row['userID']]=[itemid_2_index[row['assessmentItemID']]]

with open('test.txt','w',encoding='UTF-8') as f:
    for code,name in test_data_dic.items():
        f.write(f'{code}')
        for i in range(len(name)):
            f.write(f' {name[i]}')
        f.write(f'\n')

In [133]:
# testilist 만들기
test_data_list2=[]
for index, row in test_data.iterrows():
    test_data_list2.append(itemid_2_index[row['assessmentItemID']])
a=pd.DataFrame(test_data_list2)
with open('test.txt','w',encoding='UTF-8') as f:
    for code,name in test_data_dic.items():
        f.write(f'{code}')
        for i in range(len(name)):
            f.write(f' {name[i]}')
        f.write(f'\n')
a.to_csv(os.path.join('testilist.csv'), index=False)

In [103]:
user_mem=[]
for index, row in test_data.iterrows():
    user_mem.append(int(row['userID']))
user_mem=pd.DataFrame(user_mem)
user_mem.to_csv(os.path.join('userilist.csv'), index=False)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
1035,3,A050133008,A050000133,-1,2020-10-26 13:13:57,5289
1706,4,A070146008,A070000146,-1,2020-12-27 02:47:54,9080
3023,13,A070111008,A070000111,-1,2020-12-27 04:35:09,9660
4283,17,A090064006,A090000064,-1,2020-10-30 05:48:37,2611
4670,26,A060135007,A060000135,-1,2020-10-23 11:44:18,1422
...,...,...,...,...,...,...
260052,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615
260067,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636
260082,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402


In [140]:
user_mem

,0
0,3
1,4
2,13
3,17
4,26
...,...
739,7395
740,7404
741,7416
742,7417


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
260105,7439,A040197003,A040000197,1,2020-08-21 07:36:11,2132
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244


In [151]:
pn_train_data = data[data.answerCode >= 0]

In [152]:
pn_train_data_dic={}
for index, row in pn_train_data.iterrows():
    temp=str(itemid_2_index[row['assessmentItemID']])
    temp2=str(row['answerCode'])
    IDcode=int(temp+temp2)
    if row['userID'] in pn_train_data_dic:
        pn_train_data_dic[row['userID']].append(IDcode)
    else:
        pn_train_data_dic[row['userID']]=[IDcode]
with open('pn_train.txt','w',encoding='UTF-8') as f:
    for code, name in pn_train_data_dic.items():
        f.write(f'{code}')
        for i in range(len(name)):
            f.write(f' {name[i]}')
        f.write(f'\n')

In [154]:
neg_train_data = data[data.answerCode == 0]

In [156]:
neg_train_data_dic={}
for index, row in neg_train_data.iterrows():
    temp=str(itemid_2_index[row['assessmentItemID']])
    IDcode=int(temp)
    if row['userID'] in neg_train_data_dic:
        neg_train_data_dic[row['userID']].append(IDcode)
    else:
        neg_train_data_dic[row['userID']]=[IDcode]
with open('neg_train.txt','w',encoding='UTF-8') as f:
    for code, name in neg_train_data_dic.items():
        f.write(f'{code}')
        for i in range(len(name)):
            f.write(f' {name[i]}')
        f.write(f'\n')